<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train9_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:

train9 = pd.read_csv(r'/content/train_data9.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [ ]:

train9['Target'] = train9['Target'].replace(['Others'],'Invalid')


In [ ]:
train9['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [ ]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train9_Xtfidf = tfidf_vect.fit_transform(train9['Sentence'])


In [ ]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [ ]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [ ]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [ ]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [ ]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [ ]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train9_Xtfidf,train9['Target'],test_size=0.26,random_state=42)
support = RandomForestClassifier()
clf =support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6504424778761062
                  precision    recall  f1-score   support

        Analysis       0.49      0.35      0.41       127
      Conclusion       0.64      0.33      0.44        48
           Facts       0.72      0.87      0.79       476
         Invalid       0.49      0.53      0.51       131
           Issue       0.54      0.38      0.45        52
Rule/Law/Holding       0.73      0.31      0.44        70

        accuracy                           0.65       904
       macro avg       0.60      0.47      0.51       904
    weighted avg       0.64      0.65      0.63       904



In [ ]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 0, 3, 3, 2, 2, 1, 3, 0, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 1, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2]
[0.9972440293444291, 0.9964979025035215, 0.9961330968955769, 0.9983724526346707, 0.9986315320614843, 0.9920193987897329, 0.9820093866955332, 0.9986031131101668, 0.9904676927229271, 0.9977494900405155, 0.9922395398431559, 0.9843102167775232, 0.995611843390831, 0.996991772546113, 0.9932791838053627, 0.9887684357766177, 0.9958874418552041, 0.9960255929111506, 0.9961606719784224, 0.9986450003667924, 0.9963860124674282, 0.9972299582897722, 0.9958456151017819, 0.9990950173917988, 0.9982844842565749, 0.9952543541117063, 0.9964282111708676, 0.9925750414184288, 0.9926998399059127, 0.9876768519284489, 0.9913922054805677, 0.9922176632859185, 0.9959266478097593, 0.99638816106161, 0.9824234874513426, 0.9973332301473028, 0.9963804181235976, 0.9927353315762582, 0.9965979433928385, 0.9880508493848118, 0.9984792800011134, 0.995444

In [ ]:
unlabel_1=unlabel_1.loc[pos,:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
1,466,additional moral justification may due complai...,Analysis
2,525,85 22 sw 140 wherein accused acting upon advic...,Rule/Law/Holding
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts


In [ ]:
frame_1 = [train9,unlabel_1]
train9_1 = pd.concat(frame_1)
len(train9_1)

3534

In [ ]:
x_train_1 = tfidf_vect.transform(train9_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train9_1['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.652050919377652
                  precision    recall  f1-score   support

        Analysis       0.36      0.38      0.37        92
      Conclusion       0.67      0.40      0.50        30
           Facts       0.75      0.83      0.79       387
         Invalid       0.55      0.57      0.56       107
           Issue       0.64      0.38      0.48        42
Rule/Law/Holding       0.52      0.29      0.37        49

        accuracy                           0.65       707
       macro avg       0.58      0.48      0.51       707
    weighted avg       0.64      0.65      0.64       707



In [ ]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 5, 3, 5, 2]
[0.9978674574238234, 0.9988537372132273, 0.9896424866944035, 0.9919786824839953, 0.9978220678483234, 0.9984889264176025, 0.9966575022416814, 0.9986893504729666, 0.9904502792734791, 0.9984631946171051, 0.995762900939057, 0.9851592272689178, 0.9891720062454739, 0.9965336873507544, 0.9974080158399845, 0.9914273911365082, 0.9987836246971236, 0.9864358488471147, 0.9987426058269998, 0.9984677750039936, 0.9882758897218279, 0.9967362941152444, 0.9945022266293827, 0.9965297801835389, 0.9952023115762496, 0.9987836246971236, 0.9907172385594547, 0.9986962337146392, 0.9988433756467976, 0.9841147777531081, 0.9982574923471733, 0.9987391889064543, 0.9921091589791899, 0.9947379570066978, 0.9945910677726814, 0.9966154311545249, 0.9927077065412296, 0.9985898138942831, 0.9985427879920199, 0.9955903718902918, 0.9958959947970004, 0.997426196590502, 0.9989773853896411, 0.

In [ ]:
unlabel_2 = unlabel_2.loc[pos,:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
5,2278,evidence state appellant shows appellant alleg...,Analysis
6,570,appellant performed work asked 7500 money ther...,Facts


In [ ]:
frame_2 = [train9_1,unlabel_2]
train9_2 = pd.concat(frame_2)
len(train9_2)

3580

In [ ]:
x_train_2 = tfidf_vect.transform(train9_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train9_2['Target'],test_size=0.27,random_state=2)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6349534643226473
                  precision    recall  f1-score   support

        Analysis       0.50      0.33      0.40       150
      Conclusion       0.75      0.41      0.53        44
           Facts       0.68      0.88      0.77       496
         Invalid       0.53      0.47      0.50       161
           Issue       0.64      0.35      0.46        51
Rule/Law/Holding       0.51      0.28      0.36        65

        accuracy                           0.63       967
       macro avg       0.60      0.45      0.50       967
    weighted avg       0.62      0.63      0.61       967



In [ ]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 3, 3, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, 2, 3, 1, 5, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 0, 3, 0, 2, 2, 4, 2, 2, 2, 2, 2]
[0.9973206719663372, 0.996293923396472, 0.9974769433466453, 0.997168002964363, 0.9933682984615817, 0.9913476520322057, 0.998487469068388, 0.9982510625975011, 0.9932126818994291, 0.9857628927560089, 0.9988040151840573, 0.9987730568507099, 0.9988977412751666, 0.9984875094100456, 0.9957381155461976, 0.998293745236125, 0.9862284989058661, 0.9968898602869556, 0.9969274928480397, 0.9987807045979098, 0.987734153111402, 0.9800316626381685, 0.9985354149453538, 0.9982767195974864, 0.9937563848704373, 0.9960085747731514, 0.9968342459443403, 0.998668965338729, 0.9955470328882898, 0.9937438163453713, 0.9959329319288968, 0.9955722754684907, 0.9983836229973884, 0.997103745490932, 0.987224705484115, 0.9985158038203968, 0.9948852678730941, 0.9938566137705038, 0.9983585682428157, 0.9978927770725301, 0.998

In [ ]:
unlabel_3=unlabel_3.loc[pos,:]
frame_3 = [train9_2,unlabel_3]
train9_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train9_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train9_3['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6465753424657534
                  precision    recall  f1-score   support

        Analysis       0.38      0.32      0.34        98
      Conclusion       0.63      0.45      0.52        38
           Facts       0.72      0.87      0.79       377
         Invalid       0.56      0.54      0.55       109
           Issue       0.58      0.38      0.46        48
Rule/Law/Holding       0.66      0.32      0.43        60

        accuracy                           0.65       730
       macro avg       0.59      0.48      0.51       730
    weighted avg       0.63      0.65      0.63       730



In [ ]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 2, 0, 3, 2, 5, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 0, 2, 2, 5, 3, 0, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2]
[0.9981630781351405, 0.9893841826553714, 0.9823718362872865, 0.9809877662834804, 0.9983357823105012, 0.9956624493720856, 0.9985373622137133, 0.9948074337135837, 0.987696618533047, 0.9989399109983386, 0.9986890472085322, 0.9944735065406899, 0.9906694216603605, 0.9985504906164662, 0.9983283125243047, 0.9854630516119373, 0.9988917445391199, 0.9984506875678912, 0.9980864867918161, 0.9976879624838597, 0.9932512007292349, 0.9985400687079422, 0.9983281734431787, 0.9986470012562021, 0.9970801817679793, 0.9954858045563438, 0.9987973038258394, 0.998459051225272, 0.99791294662026, 0.9984708225910747, 0.9905784938423785, 0.9987567928530673, 0.996285821262385, 0.9948550613222723, 0.9949096304185612, 0.990815225734065, 0.9965175884674856, 0.9970676072580452, 0.985454887858881, 0.9950259096800607, 0.9987863302095944, 0.

In [ ]:
unlabel_4=unlabel_4.loc[pos,:]

In [ ]:
frame_4 = [train9_3,unlabel_4]
train9_4 = pd.concat(frame_4)
len(train9_4)
x_train_4 = tfidf_vect.transform(train9_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train9_4['Target'],test_size=0.27,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6586826347305389
                  precision    recall  f1-score   support

        Analysis       0.46      0.29      0.36       143
      Conclusion       0.66      0.45      0.53        56
           Facts       0.72      0.91      0.80       530
         Invalid       0.56      0.50      0.53       150
           Issue       0.57      0.38      0.45        56
Rule/Law/Holding       0.50      0.19      0.28        67

        accuracy                           0.66      1002
       macro avg       0.58      0.45      0.49      1002
    weighted avg       0.63      0.66      0.63      1002



In [ ]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 2, 3, 3, 2, 3, 1, 4, 2, 5, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 3, 3, 2, 3, 2, 2, 2, 0, 2, 2, 2, 3, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 0, 2, 3, 2, 2]
[0.9908100571622775, 0.9989287464961317, 0.9962174598248675, 0.9929546147010921, 0.9897603575669376, 0.9928925265912741, 0.9956008085344945, 0.9919348015058187, 0.9978250755399719, 0.9957068955790697, 0.9988351149400142, 0.9896506418453791, 0.9984209069675626, 0.9979374079384161, 0.9823401410806418, 0.9972643986347364, 0.9929895022003975, 0.9966039120379129, 0.9892052242629548, 0.9960177872213705, 0.989060198728218, 0.998944843386434, 0.9961280081380425, 0.9982265755392767, 0.9954752182643649, 0.9921142806905165, 0.9967002653608105, 0.998944843386434, 0.998747892797089, 0.9949544138170232, 0.9929546147010921, 0.9974771845648147, 0.9984283572722563, 0.9948567813225633, 0.9983181473559503, 0.9948835310699173, 0.989060198728218, 0.9969117533691779, 0.9976760666196193, 0.9975962421318979, 0.997988003028656

In [ ]:
unlabel_5 = unlabel_5.loc[pos,:]

In [ ]:
frame_5 = [train9_4,unlabel_5]
train9_5 = pd.concat(frame_5)
len(train9_5)
x_train_5 = tfidf_vect.transform(train9_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train9_5['Target'],test_size=0.27,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6669941060903732
                  precision    recall  f1-score   support

        Analysis       0.49      0.35      0.40       142
      Conclusion       0.83      0.46      0.60        54
           Facts       0.72      0.89      0.80       543
         Invalid       0.51      0.55      0.53       143
           Issue       0.62      0.43      0.51        54
Rule/Law/Holding       0.71      0.27      0.39        82

        accuracy                           0.67      1018
       macro avg       0.65      0.49      0.54      1018
    weighted avg       0.66      0.67      0.65      1018



In [ ]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.72552783109405


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.6979592702484215, 0.5876939609689512, 0.6257567597350074, None)
